In [1]:
import sys
sys.path.append('C:/Users/mdmur/OneDrive/Pictures/MohasProject/venv/Lib/site-packages')

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor

%matplotlib inline

In [3]:
file_path = 'output/all_data.csv'
df = pd.read_csv(file_path)

In [4]:
df.sample(5)

,order_id,product,quantity_ordered,price_each,order_date,month,total_sales,total_quantity_sold_all_months
38177,303956,AAA Batteries (4-pack),1,2.99,2019-12-10 10:37:00,December,2.99,30487
114172,173023,Lightning Charging Cable,2,14.95,2019-03-09 09:49:00,March,29.90,22841
59302,155312,AAA Batteries (4-pack),1,2.99,2019-02-23 19:48:00,February,2.99,30487
79506,226747,Lightning Charging Cable,1,14.95,2019-07-01 17:34:00,July,14.95,22841
82744,229920,Bose SoundSport Headphones,1,99.99,2019-07-28 11:20:00,July,99.99,13236


In [5]:
total_quantity_sold = df.groupby(['product', 'month'])['quantity_ordered'].sum().reset_index()
total_quantity_sold.rename(columns={'quantity_ordered': 'quantity_sold_that_month'}, inplace=True)

In [6]:
df = pd.merge(df, total_quantity_sold, on=['product', 'month'], how='left')

In [7]:
df = df.sort_values(by='order_date')
df.head()

,order_id,product,quantity_ordered,price_each,order_date,month,total_sales,total_quantity_sold_all_months,quantity_sold_that_month
72308,147268,Wired Headphones,1,11.99,2019-01-01 03:07:00,January,11.99,20201,1057
73104,148041,USB-C Charging Cable,1,11.95,2019-01-01 03:40:00,January,11.95,23555,1265
74423,149343,Apple Airpods Headphones,1,150.00,2019-01-01 04:56:00,January,150.00,15383,798
75060,149964,AAA Batteries (4-pack),1,2.99,2019-01-01 05:53:00,January,2.99,30487,1562
74430,149350,USB-C Charging Cable,2,11.95,2019-01-01 06:03:00,January,23.90,23555,1265


In [8]:
df['order_date'] = pd.to_datetime(df['order_date'])

In [9]:
df = df.sort_values(by=['product', 'order_date'])

In [10]:
month_order = {'January': 1, 'February': 2, 'March': 3, 'April': 4, 'May': 5, 'June': 6,
               'July': 7, 'August': 8, 'September': 9, 'October': 10, 'November': 11, 'December': 12}
df['month_num'] = df['month'].map(month_order)

previous_month_quantity = {}
next_month_quantity = {}

def get_previous_month_quantity(row):
    product = row['product']
    previous_month = row['month_num'] - 1 if row['month_num'] > 1 else 12
    return previous_month_quantity.get((product, previous_month), 0)

def get_next_month_quantity(row):
    product = row['product']
    next_month = row['month_num'] + 1 if row['month_num'] < 12 else 1
    return next_month_quantity.get((product, next_month), 0)

for index, row in df.iterrows():
    product = row['product']
    month = row['month_num']
    quantity_sold = row['quantity_sold_that_month']
    
    previous_month = month - 1 
    previous_month_quantity[(product, month)] = quantity_sold
    
    next_month = month + 1  
    next_month_quantity[(product, month)] = quantity_sold

df['quantity_sold_last_month'] = df.apply(get_previous_month_quantity, axis=1)
df['quantity_sold_next_month'] = df.apply(get_next_month_quantity, axis=1)


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 182735 entries, 75136 to 31921
Data columns (total 12 columns):
 #   Column                          Non-Null Count   Dtype         
---  ------                          --------------   -----         
 0   order_id                        182735 non-null  int64         
 1   product                         182735 non-null  object        
 2   quantity_ordered                182735 non-null  int64         
 3   price_each                      182735 non-null  float64       
 4   order_date                      182735 non-null  datetime64[ns]
 5   month                           182735 non-null  object        
 6   total_sales                     182735 non-null  float64       
 7   total_quantity_sold_all_months  182735 non-null  int64         
 8   quantity_sold_that_month        182735 non-null  int64         
 9   month_num                       182735 non-null  int64         
 10  quantity_sold_last_month        182735 non-null  int64    

In [12]:
df.isna().value_counts()

order_id  product  quantity_ordered  price_each  order_date  month  total_sales  total_quantity_sold_all_months  quantity_sold_that_month  month_num  quantity_sold_last_month  quantity_sold_next_month
False     False    False             False       False       False  False        False                           False                     False      False                     False                       182735
Name: count, dtype: int64

In [14]:
df = df.dropna()

In [15]:
df = df.drop(['month_num'], axis=1)

In [16]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
df['product_encoded'] = label_encoder.fit_transform(df['product'])

In [17]:
features = ['product_encoded', 'quantity_sold_last_month', 'quantity_sold_that_month']
target = 'quantity_sold_next_month'

In [18]:
X_train, X_test, y_train, y_test = train_test_split(df[features], df[target], test_size=0.2, random_state=42)

In [19]:
model = RandomForestRegressor(n_estimators=100, random_state=42)

In [20]:
model.fit(X_train, y_train)

RandomForestRegressor(random_state=42)

In [22]:
import joblib

joblib.dump(model, 'simprandom_r_mlV001.joblib')

['simprandom_r_mlV001.joblib']

In [32]:
user_product = input("Enter the product: ")
user_quantity_sold_last_month = int(input("Enter the quantity sold last month: "))
quantity_sold_this_month = int(input("Enter the quantity sold this month: "))

user_product_encoded = label_encoder.transform([user_product])[0]

Enter the product:  AAA Batteries (4-pack)
Enter the quantity sold last month:  2895
Enter the quantity sold this month:  2875


In [33]:
user_input = pd.DataFrame({'product_encoded': [user_product_encoded],
                            'quantity_sold_last_month': [user_quantity_sold_last_month],
                            'quantity_sold_that_month': [quantity_sold_this_month]})

predicted_quantity_sold_next_month = model.predict(user_input)

print(f"Predicted quantity sold next month: {predicted_quantity_sold_next_month[0]}")

Predicted quantity sold next month: 2114.0


In [27]:
df = df.drop(['quantity_sold_next_month'], axis=1)

In [28]:
df.to_csv('./output/visuialising1.csv', index=False)

In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 182735 entries, 75136 to 31921
Data columns (total 11 columns):
 #   Column                          Non-Null Count   Dtype         
---  ------                          --------------   -----         
 0   order_id                        182735 non-null  int64         
 1   product                         182735 non-null  object        
 2   quantity_ordered                182735 non-null  int64         
 3   price_each                      182735 non-null  float64       
 4   order_date                      182735 non-null  datetime64[ns]
 5   month                           182735 non-null  object        
 6   total_sales                     182735 non-null  float64       
 7   total_quantity_sold_all_months  182735 non-null  int64         
 8   quantity_sold_that_month        182735 non-null  int64         
 9   quantity_sold_last_month        182735 non-null  int64         
 10  product_encoded                 182735 non-null  int32    